In [142]:
! pip install lxml
! pip install html5lib

In [52]:
from urllib.request import urlopen
from urllib import request
from bs4 import BeautifulSoup
import bs4
import re
import numpy as np
import requests
from tika import parser as tkparser
import lxml
from selenium import webdriver
import time

In [2]:
def get_query_links(bs_object):
    query_links = bs_object.find('div',{'id' : 'mainWrapper'}).find('table', {'id' : 'ctl11_ctl01_ctl00_KMbase_table'}).find_all('li')
    return query_links

In [3]:
def get_filtered_links(query_links):
    query_links_filtered =[]
    for idx, link in enumerate(query_links) :
        refined = str(link).strip('<li>').strip('</li>').strip()
    #     print(refined)
        if re.compile('<a').match(refined):
            splited = refined.split('>')[0].strip('<a href=').strip('"')
            if re.compile('http').match(splited):
                continue
            else:
                query_links_filtered.append(splited)   
    return np.unique(query_links_filtered)

In [4]:
def get_query_bs( filtered_link, basic_url = 'http://kmbase.medric.or.kr',):
    query_addr = basic_url + filtered_link.replace('amp;', '')
    query_html = urlopen(query_addr)
    query_bs = BeautifulSoup(query_html, 'html.parser')
    journal_info = query_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_journal_info'}).select('a')[0].get('href')
    return query_bs, journal_info

In [ ]:
# linkout_addr = query_bs.find('span',{'id' : 'ctl11_ctl01_ctl00_fulltextlinkout'}).select('a')[1].get('href')
# linkout_html = urlopen(linkout_addr)
# linkout_bs = BeautifulSoup(linkout_html, 'html.parser')

In [60]:
basic_url = 'http://kmbase.medric.or.kr'
# html = urlopen('http://kmbase.medric.or.kr/Main.aspx?menu=01&d=SEARCH&s=NORMAL&kw=%b4%eb%bb%e7%c1%f5%c8%c4%b1%ba')
# search_bs = BeautifulSoup(html, 'html.parser')



In [61]:
n_papers = int(search_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_total_count'}).find_all('font')[1].get_text())
n_views = 20 
max_page= n_papers / n_views
basic_url = 'http://kmbase.medric.or.kr'

In [44]:
browser = webdriver.Chrome('C:/Users/NB-0242/jupyter/PHR/chromedriver.exe')
browser.get('http://kmbase.medric.or.kr/Main.aspx?menu=01&d=SEARCH&s=NORMAL&kw=%b4%eb%bb%e7%c1%f5%c8%c4%b1%ba')

In [68]:
for i in range(2,21):
    browser.find_element_by_xpath('//*[@id="ctl11_ctl01_ctl00_KMGridList"]/tbody/tr[22]/td/table/tbody/tr/td[{}]/a'.format(i)).click()
#     driver.implicitly_wait(100) 
    time.sleep(5)
    search_html = browser.page_source
    search_bs = BeautifulSoup(search_html, 'html.parser')
    search_links = get_query_links(search_bs)
    search_links_filtered = get_filtered_links(search_links)
    
    for i in range(len(search_links_filtered)):
        query_addr = basic_url + str(search_links_filtered[i]).replace('amp;', '')
        query_html = urlopen(query_addr)
        query_bs = BeautifulSoup(query_html, 'html.parser')
        journal_info = query_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_journal_info'}).select('a')[0].get('href')
        title = query_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_title_kor'}).get_text()

        print('journal info : ', journal_info)
        print('{}th ,query addr : {} '.format(i, query_addr))

        linkout_bs = get_linkout_bs(query_bs)
        if linkout_bs ==None:
            continue

        base_url, pdf_link = linkout_checker(linkout_bs, journal_info)
        if pdf_link==None:
            continue

        pdf_link = base_url + pdf_link
        parsed = tkparser.from_buffer(urlopen(pdf_link), xmlContent=True)
        pdf_bs = BeautifulSoup(parsed['content'], 'lxml')
        text_list = pdf_crwaling(pdf_bs)
        text_refined= pdf_refining(text_list)
        save_paper_(title, text_refined)

journal info :  http://ocean.kisti.re.kr/IS_mvpopo001P.do?method=multMain&poid=kscn&free
0th ,query addr : http://kmbase.medric.or.kr/Main.aspx?menu=01&d=KMBASE&m=VIEW&i=0123420190240030232  
papers/국민건강영양조사(2013-2014년)_자료에_근거한_가구원수별_구성원의_영양상태_및_대사증후군_유병율_평가.txt
journal info :  https://kjan.or.kr/index.php?body=archive
1th ,query addr : http://kmbase.medric.or.kr/Main.aspx?menu=01&d=KMBASE&m=VIEW&i=0367020190310020202  
papers/성인초기_대상자에서_지속성_피로에_따른_스트레스와_수면의_질_및_대사증후군_차이.txt
journal info :  https://kjan.or.kr/index.php?body=archive
2th ,query addr : http://kmbase.medric.or.kr/Main.aspx?menu=01&d=KMBASE&m=VIEW&i=0367020190310040449  
pdf_link unavailable
journal info :  http://www.ksphn.or.kr/02_sub/2c_sub01.php
3th ,query addr : http://kmbase.medric.or.kr/Main.aspx?menu=01&d=KMBASE&m=VIEW&i=0379720190330010020  
journal info :  http://ocean.kisti.re.kr/IS_mvpopo210L.do?method=list&poid=kshpa&kojic=BGHJBH&sFree=
4th ,query addr : http://kmbase.medric.or.kr/Main.aspx?menu=01&d=KMBASE&m=V

IndexError: list index out of range

In [76]:
second_bs = BeautifulSoup(urlopen('http://kmbase.medric.or.kr/Main.aspx?menu=01&d=KMBASE&m=VIEW&i=0367020190310040449 '), 'html.parser')
journal_info = second_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_journal_info'}).select('a')[0].get('href')

In [73]:
get_linkout_bs(second_bs)

linkout button try


<!DOCTYPE html>

<html lang="en" xmlns:mml="http://www.w3.org/1998/Math/MathML">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="https://kjan.or.kr/images/header.png" property="og:image"/>
<title>:: KJAN :: Korean Journal of Adult Nursing</title>
<link href="css/bootstrap.min.css" rel="stylesheet"/>
<link href="css/default.css?1162239046" rel="stylesheet"/>
<link href="css/font-awesome.min.css" rel="stylesheet"/>
<link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet" type="text/css"/>
<link href="https://use.fontawesome.com/releases/v5.0.6/css/all.css" rel="stylesheet"/>
<link href="https://fonts.googleapis.com/css?family=Noto+Serif:400,400italic,700,700italic&amp;subset=latin,greek,latin-ext,greek-ext,cyrillic,vietnamese,cyrillic-ext" rel="stylesheet" type="text/css"/>
<link href

In [77]:
linkout_addr = second_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_fulltextlinkout'}).a.get('href')
linkout_html = urlopen(linkout_addr)
linkout_bs = BeautifulSoup(linkout_html, 'html.parser')

In [78]:
linkout_checker(linkout_bs, journal_info)

pdf_link unavailable


(None, None)

In [79]:
journal_info

'https://kjan.or.kr/index.php?body=archive'

In [9]:
journal_list = []
for i in range(len(query_links_filtered)):
#     print(i)
    query_bs, journal_info = get_query_bs(filtered_link = query_links_filtered[i])
    linkout_bs = get_linkout_bs(query_bs, journal_info)
    journal_list.append(journal_info)


In [57]:
np.unique(journal_list)

array(['http://jkfn.kfn.or.kr/',
       'http://ocean.kisti.re.kr/IS_mvpopo001P.do?method=multMain&poid=kns&free=',
       'http://ocean.kisti.re.kr/IS_mvpopo001P.do?method=multMain&poid=kscn&free',
       'http://www.jkaoh.org/',
       'http://www.kjcls.org/journal/archives.html',
       'http://www.kjfp.or.kr/journal/archives.html',
       'http://www.kjohn.or.kr/journal/archives.html',
       'http://www.ksbtp.org/', 'http://www.sportsmed.or.kr/',
       'https://e-jhis.org/articles/archive.php',
       'https://jkan.or.kr/index.php?body=archive',
       'https://kjan.or.kr/index.php?body=archive'], dtype='<U72')

In [54]:
def linkout_checker(linkout_bs, journal_info):

    
    # pdf link out check
    if journal_info == 'http://www.kjfp.or.kr/journal/archives.html':
        
        base_url = 'https://www.kjfp.or.kr'
        sub1 = re.compile(".*replace").sub("", linkout_bs.get_text()).strip("('")
        sub2 = re.compile("'.*", re.DOTALL).sub( "",sub1)
        linkout_addr2 = base_url + sub2
        linkout_html2 = urlopen(linkout_addr2)
        linkout_bs2 = BeautifulSoup(linkout_html2, 'html.parser')
        pdf_link = linkout_bs2.find('dd',{'style' : 'color: #A52A54; line-height:20px;'}).select('a')[0].get('href')
        return base_url, pdf_link
    
    
    elif journal_info == 'http://www.kjcls.org/journal/archives.html':
        
#         linkout_html = urlopen(linkout_addr)
#         linkout_bs = BeautifulSoup(linkout_html, 'html.parser')
        pdf_link = linkout_bs.find('div',{'class' : 'sub_rig02'}).dd.a.get('href')
        return '' , pdf_link
        
    elif journal_info == 'https://e-jhis.org/articles/archive.php':
        
        base_url = 'https://e-jhis.org/upload/pdf/'
#         linkout_html = urlopen(linkout_addr)
#         linkout_bs = BeautifulSoup(linkout_html, 'html.parser')
        pdf_name= [re.compile('.*[.]pdf').search(x) for x in str(linkout_bs.find('td',{'style': 'padding-left:10px;padding-top:6px'}).a).split() if re.compile('.*[.]pdf').search(x) != None][0]
        pdf_link = re.compile('"').sub('', pdf_name.string).replace(")'", '')
        
        return base_url, pdf_link
        

    
    
    elif  journal_info == 'https://kjan.or.kr/index.php?body=archive':
        
      
    
    elif journal_info =='http://www.ksbtp.org/':
    print('ksbtp not support yet')

    return None, None
    
    else : 
        base_url = 'https://synapse.koreamed.org/'
        try:
            pdf_link = linkout_bs.find('div', {'id': 'ArticleContentsTopIconBar'}).select('a')[2].get('href')
            return base_url, pdf_link

        except: #cross-ref 로 가는 경우 & 404 not found

            try :
                linkout2 = linkout_bs.find('div', {'id': 'prime'}).select('a')[0].get('href')
                linkout2_html = urlopen(linkout2)
                linkout2_bs = BeautifulSoup(linkout2_html, 'html.parser')
                pdf_link = linkout2_bs.find('div', {'id': 'ArticleContentsTopIconBar'}).select('a')[2].get('href')
                return base_url, pdf_link
            except :
#                 print('pdf_link unavailable')
                return None, None

In [55]:
def save_paper_(title, text_refined, directory_path='papers/'):
    title = re.compile(r':| |/').sub('_',title, ).replace('?','_')
    save_path = directory_path + title + '.txt'
    print(save_path)

    with open(save_path, 'w', encoding='utf-8') as f:
        f.write(title + '\n')
        for i in range(len(text_refined)):
            f.write(text_refined[i])

In [72]:
def get_linkout_bs(query_bs):
    
    linkout_addr = query_bs.find('span',{'id' : 'ctl11_ctl01_ctl00_doia'}).select('a')[0].get('href')
    if not bool(re.compile('doi.org').search(linkout_addr)):
        linkout_addr = linkout_addr.replace('https://','')
        linkout_addr = 'https://doi.org/' + linkout_addr
        
    # 404 not found check
    # doi link
    try : 
        linkout_html = urlopen(linkout_addr)
        linkout_bs = BeautifulSoup(linkout_html, 'html.parser')
        print('doi link success')
        return linkout_bs
        
    except : # link out button try
        print('linkout button try')
        try : 
            linkout_addr = query_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_fulltextlinkout'}).a.get('href')
            linkout_html = urlopen(linkout_addr)
            linkout_bs = BeautifulSoup(linkout_html, 'html.parser')
            return linkout_bs
        
        except:
            print('linkout unavailable')
            return None

13
journal info :  http://www.kjfp.or.kr/journal/archives.html
query addr :  http://kmbase.medric.or.kr/Main.aspx?menu=01&d=KMBASE&m=VIEW&i=1039620200100010044 
doi link success
papers/한국_성인에서_10년간_대사증후군_유병률과_위험요인의_변화__국민건강영양조사(2008_2017)_자료를_이용하여.txt
14
journal info :  http://www.kjcls.org/journal/archives.html
query addr :  http://kmbase.medric.or.kr/Main.aspx?menu=01&d=KMBASE&m=VIEW&i=1101420190510010042 
doi link success
papers/건강한_성인_남성의_대사증후군_위험_예측인자로서_혈청_요산__크레아티닌_비율.txt
15
journal info :  http://www.kjcls.org/journal/archives.html
query addr :  http://kmbase.medric.or.kr/Main.aspx?menu=01&d=KMBASE&m=VIEW&i=1101420190510030277 
doi link success
papers/건강한_노인에서_심장-발목_혈관지수로_측정한_동맥_경직도와체질량_지수간의_상관관계.txt
16
journal info :  http://jkfn.kfn.or.kr/
query addr :  http://kmbase.medric.or.kr/Main.aspx?menu=01&d=KMBASE&m=VIEW&i=1134820190480080802 
doi link success
pdf_link unavailable
17
journal info :  https://e-jhis.org/articles/archive.php
query addr :  http://kmbase.medric.or.kr/Main.

In [608]:
query_addr = basic_url + str(query_links_filtered[14]).replace('amp;', '')
query_html = urlopen(query_addr)
query_bs = BeautifulSoup(query_html, 'html.parser')
journal_info = query_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_journal_info'}).select('a')[0].get('href')
title = query_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_title_kor'}).get_text()

In [610]:
print('journal info : ', journal_info)
print('query addr : ', query_addr)

linkout_bs = get_linkout_bs(query_bs)
# # if linkout_bs ==None:
# #     continue

base_url, pdf_link = linkout_checker(linkout_bs, journal_info)
# pdf_link = base_url + pdf_link
# parsed = tkparser.from_buffer(urlopen(pdf_link), xmlContent=True)
# pdf_bs = BeautifulSoup(parsed['content'], 'lxml')
# text_list = pdf_crwaling(pdf_bs)
# text_refined= pdf_refining(text_list)
# save_paper_(title, text_refined)

journal info :  http://www.kjcls.org/journal/archives.html
query addr :  http://kmbase.medric.or.kr/Main.aspx?menu=01&d=KMBASE&m=VIEW&i=1101420190510010042 
doi link success


In [578]:
# E-JHIS
base_url, pdf_link = linkout_checker(linkout_bs, journal_info)
pdf_link = base_url + pdf_link
parsed = tkparser.from_buffer(urlopen(pdf_link), xmlContent=True)
pdf_bs = BeautifulSoup(parsed['content'], 'lxml')
text_list = pdf_crwaling(pdf_bs)
text_refined= pdf_refining(text_list)

In [579]:
save_paper_(title, text_refined)

papers/우리나라_성인의_근육량과_대사증후군_구성요소와의_관련성.txt


In [ ]:
# JKFN
# PASS

In [472]:
# KJCLS
_, pdf_link = linkout_checker(linkout_addr)
parsed = tkparser.from_buffer(urlopen(pdf_link), xmlContent=True)
pdf_bs = BeautifulSoup(parsed['content'], 'lxml')
text_list = pdf_crwaling(pdf_bs)
text_refined= pdf_refining(text_list)

In [473]:
save_paper_(title, text_refined)

papers/건강한_노인에서_심장-발목_혈관지수로_측정한_동맥_경직도와체질량_지수간의_상관관계.txt


In [426]:

# KJFP
base_url, pdf_link = linkout_checker(linkout_addr)

if pdf_link != None:
    pdf_link = base_url + pdf_link
    parsed = tkparser.from_buffer(urlopen(pdf_link), xmlContent=True)
    pdf_bs = BeautifulSoup(parsed['content'], 'lxml')
    text_list = pdf_crwaling(pdf_bs)
    text_refined= pdf_refining(text_list)
    
else:
    print('pdf_link unavailable')

In [439]:
save_paper_(title, text_refined)

papers/한국_성인에서_10년간_대사증후군_유병률과_위험요인의_변화__국민건강영양조사(2008_2017)_자료를_이용하여.txt


In [294]:

# synapse 
pdf_link = linkout_checker(linkout_addr)

if pdf_link!=None:
    base_url = 'https://synapse.koreamed.org/'
    pdf_link = base_url + pdf_link
    parsed = tkparser.from_buffer(urlopen(pdf_link), xmlContent=True)
    pdf_bs = BeautifulSoup(parsed['content'], 'lxml')
    text_list = pdf_crwaling(pdf_bs)
    text_refined = pdf_refining(text_list)
    
else:
    print('pdf_link unavailable')

In [307]:
save_paper_(title, text_refined)

papers/배추김치_섭취와_대사증후군_발생률과의_관련성__한국인유전체역학조사사업의_10년_추적조사_결과.txt


> ocean.kist

In [260]:
linkout_addr = query_bs.find('span',{'id' : 'ctl11_ctl01_ctl00_doia'}).select('a')[0].get('href')
# linkout_html = urlopen(linkout_addr)
# linkout_bs = BeautifulSoup(linkout_html, 'html.parser')
pdf_link = linkout_checker(linkout_addr)

In [261]:
base_url = 'https://synapse.koreamed.org/'
pdf_link = base_url + pdf_link
parsed = tkparser.from_buffer(urlopen(pdf_link), xmlContent=True)
pdf_bs = BeautifulSoup(parsed['content'], 'lxml')

'Synapse/Data/PDFData/1124JNH/jnh-52-227.pdf'

In [270]:
text_list = pdf_crwaling(pdf_bs)

In [271]:
text_refined = pdf_refining(text_list)

> kjohn

In [253]:
linkout_addr = query_bs.find('span',{'id' : 'ctl11_ctl01_ctl00_doia'}).select('a')[0].get('href')
linkout_html = urlopen(linkout_addr)
linkout_bs = BeautifulSoup(linkout_html, 'html.parser')

HTTPError: HTTP Error 404: 

> jkan

In [228]:
linkout_html = urlopen("https://doi.org/10.4040/jkan.2019.49.6.713")

In [231]:
linkout_bs = BeautifulSoup(linkout_html, 'html.parser')

In [242]:
base_url = 'https://synapse.koreamed.org/'
pdf_link = base_url + pdf_link

In [243]:
parsed = tkparser.from_buffer(urlopen(pdf_link), xmlContent=True)
pdf_bs = BeautifulSoup(parsed['content'], 'lxml')

In [64]:
def pdf_crwaling(pdf_bs):
    # text_crwaling
    for idx, p_tag in enumerate(pdf_bs.find('body').find_all('p')):
        if re.compile('.*서.*론.*').match(str(p_tag)):
            start_idx = idx
            break
        else :
            start_idx = None
            
    for idx, p_tag in enumerate(pdf_bs.find('body').find_all('p')):
        if re.compile('.*references.*').match(str(p_tag).lower()):
            end_idx = idx
            break
        else:
            end_idx = None

    if (start_idx != None) and (end_idx != None):
        text_list = []
        p_tags = pdf_bs.find('body').find_all('p')
        for idx in range(start_idx, end_idx):
            text_list.append(p_tags[idx].get_text())

    else :
        text_list = []
        for p_tag in pdf_bs.body.find_all('p'):
            text_list.append(p_tag.get_text())
        
    return text_list

In [65]:
def pdf_refining(text_list):
    text_list_refined = []
    for sentence in text_list:
        if len(re.compile('[가-힣]').findall(sentence)) > len(sentence) * 0.3:
            text_list_refined.append(sentence.replace('\n', ''))
        else:
            text_list_refined.append(sentence)

    text_list_refined2 = []
    for idx, sentence in enumerate(text_list_refined):
        if re.compile('다[.]').search(sentence):
            m = re.compile('다[.]').search(sentence).span()[1]
            text_list_refined2.append(sentence[:m] + '\n' +sentence[m:])
        else:
            text_list_refined2.append(sentence)
            
    text_list_refined3 = []
    for idx, sentence in enumerate(text_list_refined2):
        if re.compile('다[0-9][.]').search(sentence):
    #         print(sentence)
            m = re.compile('다[0-9][.]').search(sentence).span()[1]
            text_list_refined3.append(sentence[:m-2] + sentence[m-1] +'\n' + sentence[m:])   

        elif re.compile('다[0-9]').search(sentence):
            m = re.compile('다[0-9]').search(sentence).span()[1]
            text_list_refined3.append(sentence[:m-1].join('.') + '\n' + sentence[m:])
        else:
            text_list_refined3.append(sentence)
            
    return text_list_refined3

> sportsmed

In [200]:
title = query_bs.find('span', {'id' : 'ctl11_ctl01_ctl00_title_kor'}).get_text()

In [98]:
linkout_addr = query_bs.find('span',{'id' : 'ctl11_ctl01_ctl00_doia'}).select('a')[0].get('href')
linkout_html = urlopen(linkout_addr)
linkout_bs = BeautifulSoup(linkout_html, 'html.parser')

In [102]:
pdf_link = linkout_bs.find('div', {'id' : 'ArticleContentsTopIconBar'}).select('a')[2].get('href')
base_url = 'https://synapse.koreamed.org/'
pdf_link = base_url + pdf_link

In [103]:
parsed = tkparser.from_buffer(urlopen(pdf_link), xmlContent=True)
pdf_bs = BeautifulSoup(parsed['content'], 'lxml')

In [109]:
# text_crwaling
for idx, p_tag in enumerate(pdf_bs.find('body').find_all('p')):
    if re.compile('.*서.*론.*').match(str(p_tag)):
        start_idx = idx
        break
        
for idx, p_tag in enumerate(pdf_bs.find('body').find_all('p')):
    if re.compile('.*References.*').match(str(p_tag)):
        end_idx = idx
        break
        
text_list = []
p_tags = pdf_bs.find('body').find_all('p')
for idx in range(start_idx, end_idx):
    text_list.append(p_tags[idx].get_text())

In [193]:
with open('test2.txt', 'w', encoding='utf-8') as f:
    f.write(title +'\n')
    for i in range(len(text_list_refined3)):
        f.write(text_list_refined3[i])

> jkaoh

In [11]:
linkout_addr = query_bs.find('span',{'id' : 'ctl11_ctl01_ctl00_doia'}).select('a')[0].get('href')
linkout_html = urlopen(linkout_addr)
linkout_bs = BeautifulSoup(linkout_html, 'html.parser')

In [12]:
pdf_link = linkout_bs.find('div', {'id' : 'ArticleContentsTopIconBar'}).select('a')[2].get('href')
base_url = 'https://synapse.koreamed.org/'
pdf_link = base_url + pdf_link
# request.urlretrieve(pdf_link, 'temp.pdf')

parsed = tkparser.from_buffer(urlopen(pdf_link), xmlContent=True)
pdf_bs = BeautifulSoup(parsed['content'], 'lxml')

#meta info
title = pdf_bs.find('div',{'class' : 'page'}).find_all('p')[1].get_text()
author = re.compile('\d').sub('', pdf_bs.find('div',{'class' : 'page'}).find_all('p')[2].get_text())
for p_tag in pdf_bs.find('div',{'class' : 'page'}).find_all('p'):
    if re.compile('.*Accepted.*').match(str(p_tag)):
        date = p_tag.get_text()
        
    elif re.compile('.*Key Words.*').match(str(p_tag)):
        key_words = p_tag.get_text()
         

('temp.pdf', <http.client.HTTPMessage at 0x1833889d4c8>)

In [13]:
# text_crwaling
for idx, p_tag in enumerate(pdf_bs.find('body').find_all('p')):
    if re.compile('.*서.*론.*').match(str(p_tag)):
        start_idx = idx
        break
        
for idx, p_tag in enumerate(pdf_bs.find('body').find_all('p')):
    if re.compile('.*References.*').match(str(p_tag)):
        end_idx = idx
        break
        
text_list = []
p_tags = pdf_bs.find('body').find_all('p')
for idx in range(start_idx, end_idx):
    text_list.append(p_tags[idx].get_text())